In [70]:
import pickle
import pandas as pd
from mwclient import Site
import datetime
import time
import os

In [71]:
file_name = 'politician-data-0-100'
tracker_file = file_name+'-tracker.csv' # make sure the file is in parallel to this program

base_path = 'profile-data' # to store the profile data. if the folder not present then it will create it
# no path delimiters please. don't use //
if(not os.path.isdir(base_path)):
    os.makedirs(base_path)


In [72]:
user_agent = 'Uni Koblenz-Landau student, kandhasamy@uni-koblenz.de'
wiki = Site(host='en.wikipedia.org', clients_useragent=user_agent)

In [73]:
start_time = time.time()
start_time

1483621788.168219

In [74]:
pol_tracker = None
def read_politician_tracker() : 
    global pol_tracker
    pol_tracker = pd.read_csv(tracker_file)
    pol_tracker = pol_tracker[['handle','finished_reading','time_taken_in_mins']]
read_politician_tracker()
pol_tracker.head()

,handle,finished_reading,time_taken_in_mins
0,Quincy_Timberlake,False,0.0
1,Fanny_Edelman,False,0.0
2,Stojan_Novaković,False,0.0
3,Paul_Scheffer,False,0.0
4,Ed_Robb,False,0.0


In [75]:
def get_unread_politician(pol_tracker):
    data_to_be_read = pol_tracker[pol_tracker['finished_reading'] == False]
    if(data_to_be_read.shape[0] > 0) :
        return data_to_be_read.iloc[0]['handle']
    else:
        return None
    
def write_read_politician(pol_tracker, politician,time_taken):
    # this could be improved - rather than filtering two times - get the row handle and update it
    pol_tracker.loc[pol_tracker['handle'] == politician, 'finished_reading'] = True
    pol_tracker.loc[pol_tracker['handle'] == politician, 'time_taken_in_mins'] = time_taken 

    pol_tracker.to_csv(tracker_file)
    
get_unread_politician(pol_tracker)

'Quincy_Timberlake'

In [76]:
start_time = time.time()
politician_count = 0
while True:
    # get the politician to be read
    # read them completely
    #jump on to the next politician until everything is over
    start_politician_time = time.time()
    unread_politician = get_unread_politician(pol_tracker)
    if (unread_politician):
        politician_page = wiki.pages[unread_politician]
        pol_list = {}
        # go through all the years and all months and read a single article (or the latest previous article)
        for year in range(2001,2017):
            for mon in range(1,13):
                dateObj = datetime.date(year, mon, 1)
                #startDate = dateObj.isoformat()+'T00:00:00Z'
                endDate = dateObj.isoformat()+'T23:59:59Z'
                for article in politician_page.revisions(start=endDate, prop='ids|timestamp|content',dir='older', limit=1):
                    pol_list[str(year)+'-'+str(mon)] = article
                    break
        politician_count += 1
        end_politician_time = time.time()
        print(politician_count,') '+unread_politician+' is read. '+'time taken(mins) - ',(end_politician_time - start_politician_time)/60)
        pickle.dump(pol_list,open(base_path + '/'+unread_politician,'wb'))
        write_read_politician(pol_tracker, unread_politician, (end_politician_time - start_politician_time)/60 )
    else :
        print('Hooray!!!!!!! The job is over.')
        break
end_time = time.time()
print('Total Time taken -',(end_time - start_time) / 60) 

1 ) Quincy_Timberlake is read. time taken(mins) -  0.8100204745928447
2 ) Fanny_Edelman is read. time taken(mins) -  0.7990706483523051
3 ) Stojan_Novaković is read. time taken(mins) -  0.8789964079856872
4 ) Paul_Scheffer is read. time taken(mins) -  0.7661576906840006
5 ) Ed_Robb is read. time taken(mins) -  0.7450967232386271
6 ) Nizamettin_Erkmen is read. time taken(mins) -  0.6623029510180155
7 ) Claudio_Scajola is read. time taken(mins) -  0.7038844704627991
8 ) Lawrence_Lual_Lual is read. time taken(mins) -  0.7397859930992127
9 ) Thomas_Clausen_(Louisiana) is read. time taken(mins) -  0.6808087031046549
10 ) Yang_Ti-liang is read. time taken(mins) -  0.70073535044988
11 ) Georges_Colombier is read. time taken(mins) -  0.6332885384559631
12 ) Gottlieb_Duttweiler is read. time taken(mins) -  0.6467467784881592
13 ) Mohammad_Natsir is read. time taken(mins) -  0.7488250454266866
14 ) Ján_Ševčík is read. time taken(mins) -  0.7234708944956462
15 ) Georges_Othily is read. time taken

### Just interrupt the above code, whenever you want to and run the following code just to check how much time it took

In [52]:
curr_time = time.time()
start_time

1483616594.0808723

In [53]:
politician_count

10

In [54]:
#Overall time in hrs
(curr_time - start_time) / 3600

0.1152450070116255

In [55]:
#Overall time in mins
(curr_time - start_time) / 60

6.91470042069753

In [56]:
#Average time for one profile in mins
(curr_time - start_time ) / (60 *politician_count)

0.691470042069753

In [23]:
#pol_list
#unread_politician
#pickle.dump(pol_list,open('profile-data//'+unread_politician,'wb'))
